In [48]:
import pandas as pd
from glob import glob
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.firefox.firefox_binary import FirefoxBinary
from time import sleep
import re

In [34]:
all_speechlists = glob('data/speech_lists/*.csv')

In [35]:
df_set = (pd.read_csv(f) for f in all_speechlists)
df = pd.concat(df_set, ignore_index=True)

In [36]:
no_list = [i.split('/')[-1].replace('.csv', '') for i in all_speechlists]

In [37]:
member_texts = {}

In [38]:
for no in no_list:
    member_texts[no] = []

In [40]:
len(df)

3519

In [45]:
df = df.drop_duplicates(subset=['href']).copy()

In [46]:
df.shape

(2436, 5)

In [ ]:
binary = FirefoxBinary('/Applications/Firefox_32.app/Contents/MacOS/firefox-bin')
driver = webdriver.Chrome(r'_helpers/chromedriver')

In [56]:
speech_href = df['href'].values[8]

driver.get(speech_href)
sleep(3)

# Problème: l’intervention peut être noyée parmi celles des autres, non?
# On pourrait récupérer les résumés au stade 2 et voir si ça matche

# Plus simple: on merge les duplicates, et on checke avec l’id de l’élu

transcripts = WebDriverWait(driver, 10).until(
    EC.presence_of_element_located((By.CSS_SELECTOR, '#areaDebates'))
)

doc = BeautifulSoup(transcripts.get_attribute('innerHTML'), 'html.parser')

speeches = doc.select('.pd-person-description')
print(len(speeches), 'parts found.')

member_speeches = []
for speech in speeches:
    member_href = speech.find('a', {'class': 'person-name'})
    if member_href:
        print('Nom trouvé:', member_href.text.strip(), end=' ')
    
        if member_href.get('href'):
            memberId = member_href.get('href').split('=')[-1]
            print('({})'.format(memberId), end='... ')

            if memberId in no_list:
                print('match avec le no de l’élu:', memberId)
                if len(speech.find_all('p')) > 1:
                    print('merde, il peut y avoir > 1 paragraphe => à revoir')
                member_texts[memberId].append(speech.find('p').text)
            else:
                print('Pas de match')
        else:
            print('Pas de lien sur le nom:', member_href.string.strip())


16 parts found.
Nom trouvé: Mazzone Lisa (4186)...match avec le no de l’élu: 4186
Nom trouvé: Walliser Bruno (4177)...Pas de match
Nom trouvé: Gmür-Schönenberger Andrea (4181)...Pas de match
Nom trouvé: Fehlmann Rielle Laurence (4195)...match avec le no de l’élu: 4195
Nom trouvé: Gmür-Schönenberger Andrea (4181)...Pas de match
Nom trouvé: Nordmann Roger (1279)...Pas de match
Nom trouvé: Gmür-Schönenberger Andrea (4181)...Pas de match
Nom trouvé: Sommaruga Carlo (1120)...match avec le no de l’élu: 1120
Nom trouvé: Gmür-Schönenberger Andrea (4181)...Pas de match
Nom trouvé: Genecand Benoît (4191)...match avec le no de l’élu: 4191
Nom trouvé: Gmür-Schönenberger Andrea (4181)...Pas de match
Nom trouvé: Addor Jean-Luc (4154)...match avec le no de l’élu: 4154
Nom trouvé: Gmür-Schönenberger Andrea (4181)...Pas de match


In [61]:
for key, texts in member_texts.items():
    if len(texts) > 0:
        print('>')

>
>
>
>
>
>
>
>


In [ ]:

# a merger avec bio des élus
df_speeches = pd.DataFrame(member_speeches, columns=['Texte ou intervention'])
df_speeches.to_csv('data/{}_{}.csv'.format(test_id, row['lastName']), index=False)